<a href="https://colab.research.google.com/github/ayundina/job_posts_analysis/blob/main/scrape_job_posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Scrape google job search for job posts** 

In [ ]:
%%capture
!pip install google-search-results

In [ ]:
from serpapi import GoogleSearch